# Prep, Run and Eval QAMPARI Baselines

This includes
- Use of new utils to view the retrieval results (positive)
- Use of new utils to calculate some metrics (Recall, Precision, F1)

TODO:
- It would be good to color the answer list based on whether they were found in the positive contexts.
- It would be good to visualize the top k contexts too (not just the postive ones)
- It would be good to get some metrics (maybe print then per question? but mainly, across the dataset
- Also, do matching based on how QAMPARI determined positive contexts, because for element 2 they definitely aren't by exact match.  Figure this out.

In [2]:
import json
import jsonlines
import sh

import multiqa_utils.general_utils as gu
import multiqa_utils.qampari_utils as qu
import multiqa_utils.eval_utils as eu

%load_ext autoreload
%autoreload 2

## Inspect QAMPARI provided retrieval results

They provide BM25/DPR predictions [here](https://samsam3232.github.io/qampari).

In [3]:
qmp_dwn_path = "/scratch/ddr8143/multiqa/qampari_data/qampari_downloads/"

In [4]:
print(sh.ls(qmp_dwn_path))
print(">> fid_bm25_results")
print(sh.ls(f"{qmp_dwn_path}/fid_bm25_results"))

fid_bm25_results  fid_dpr_results  rag_results

>> fid_bm25_results
full_dev_data.jsonl	  full_test_data_gold.jsonl.gz
full_dev_data_gold.jsonl  full_train_data.jsonl.gz
full_test_data.jsonl.gz



In [5]:
fid_bm25_dev = gu.readjsonl(f"{qmp_dwn_path}/fid_bm25_results/full_dev_data.jsonl")
fid_bm25_dev_gold = gu.readjsonl(f"{qmp_dwn_path}/fid_bm25_results/full_dev_data_gold.jsonl")

In [6]:
fid_bm25_dev[0].keys()

dict_keys(['id', 'question', 'answers', 'ans_mappings', 'target', 'ctxs', 'positive_ctxs'])

In [7]:
qu.print_data(fid_bm25_dev[2])

Type:                958__wikidata_comp__dev
Question:            Where did the White House Deputy Chief of Staff receive their education?
Question Keywords:   Where, did, White, House, Deputy, Chief, Staff, receive, education
Answers:             Hillcrest High School, University of Auckland, UC Berkeley School of Law, Kenyon College, Stanford University
Len pos contexts:    10
Len ctxs:            200
----------------------------------
182.5096 | Harriet Miers
    >> Harriet Ellan Miers (born August 10, 1945) is an American lawyer who served as White House Counsel to President George W. Bush from
       2005 to 2007. A member of the Republican Party since 1988, she previously served as White House Staff Secretary from 2001
       to 2003 and White House Deputy Chief of Staff for Policy from 2003 until 2005. In 2005, Miers was
       nominated by Bush to be an associate justice of the Supreme Court of the United States to replace retiring Justice Sandra Day O'Connor, but—in the
      

## Calculate Metrics on Data

**Calculating metrics on the fid_bm25_dev dataset.**

In [19]:
for kv in [20, 100, 200]:
    print(f"Performance @{kv}")
    dataset_results = eu.evaluate_dataset(fid_bm25_dev, k=kv)
    for k, v in dataset_results.items():
        if "avg" in k:
            print(f"{k + ':':25} {v*100.0:0.2f}%")
    print()

Performance @20
avg_recall:               36.18%
avg_precision:            23.56%
avg_f1:                   25.47%

Performance @100
avg_recall:               56.17%
avg_precision:            15.35%
avg_f1:                   21.06%

Performance @200
avg_recall:               62.40%
avg_precision:            12.07%
avg_f1:                   17.38%



In [17]:
eu.viz_correct_answers_context_list(fid_bm25_dev[1])

Question: Which Judge of the United States Court of Appeals for the Second Circuit works for Yale Law School?
Answers: [['Charles Edward Clark'], ['Henry Wade Rogers'], ['John M. Walker, Jr.'], ['Ralph K. Winter, Jr.'], ['Thomas Walter Swan'], ['Guido Calabresi']]
-------
Returned Answers: ['Charles Edward Clark', 'Ralph K. Winter, Jr.', 'Thomas Walter Swan', 'Guido Calabresi']
[Recall: 66.67%] 4 out of 6 in context list
[Precision: 9.00%] 18 out of 200 contexts contained an answer
